In [2]:
import mysql.connector

In [50]:
con = mysql.connector.connect(
    host="localhost", 
    port=3306, 
    user="root", 
    password="my-secret-pw",
    auth_plugin='mysql_native_password',
    database="ded_moroz_schema"
)

In [51]:
cur = con.cursor()

In [54]:
cur.execute("SELECT * FROM children")
res = cur.fetchall()
res

[(1, 'Ivan', 'Ivanov', 'm', 10, 1, 1, 1),
 (2, 'Petr', 'Petrov', 'm', 5, 1, 2, 2),
 (3, 'Andrey', 'Babkin', 'm', 21, 0, 3, 3),
 (4, 'Masha', 'Levitina', 'f', 5, 0, 4, 3),
 (5, 'Ivan', 'Bobov', 'm', 6, 1, 4, 4)]

## CRUD

Идея! Создадим дополнительную таблицу, в которой будут храниться письма детей Деду Морозу.

### Create

In [85]:
cur.execute("""CREATE TABLE IF NOT EXISTS ded_moroz_schema.letters (
               id INT NOT NULL AUTO_INCREMENT,
               child_id VARCHAR(8192),
               content VARCHAR(100),
               PRIMARY KEY (`id`));""")

cur.execute(command)
con.commit()

### Insert

In [91]:
command = ('''INSERT INTO letters (id, child_id, content) 
              VALUES (1, 1, "Дорогой Дед Мороз, я был хорошим мальчиком, поради мне подарочек")''')
cur.execute(command)
con.commit()

In [ ]:
command = ('''INSERT INTO letters (id, child_id, content) 
              VALUES (2, 2, "Дорогой Дедушка Мороз, хочу подарочек")''')
cur.execute(command)
con.commit()

In [92]:
cur.execute("SELECT * FROM letters")
res = cur.fetchall()
res

[(1, 1, 'Дорогой Дед Мороз, я был хорошим мальчиком, поради мне подарочек'),
 (2, 2, 'Дорогой Дедушка Мороз, хочу подарочек')]

### Delete

In [94]:
command = "DELETE FROM letters WHERE id = 1 "
cur.execute(command)
con.commit()

In [95]:
cur.execute("SELECT * FROM letters")
res = cur.fetchall()
res

[(2, 2, 'Дорогой Дедушка Мороз, хочу подарочек')]

### Update

In [97]:
command = "UPDATE letters SET child_id=2, content='Дорогой Дедушка Мороз, хочу мир во всем мире' WHERE id=2"
cur.execute(command)
con.commit()

In [98]:
cur.execute("SELECT * FROM letters")
res = cur.fetchall()
res

[(2, 2, 'Дорогой Дедушка Мороз, хочу мир во всем мире')]

### 1.1 Посчитать общее число детей и вывести уникальные имена
- Агрегация

In [24]:
q = ('''
    SELECT COUNT(id) FROM children;
    ''')

cur.execute(q)
data = cur.fetchall()
print(data[0][0])

5


In [74]:
q = ('''
    SELECT distinct name FROM children;
    ''')

cur.execute(q)
data = cur.fetchall()
for n in data:
    print(n[0])

Ivan
Petr
Andrey
Masha


### 2. Все дети из Тулы
- JOIN

In [20]:
q = ("""SELECT
    children.name
    , children.surname
    FROM children
    JOIN addresses
    ON children.address_id = addresses.id
    JOIN cities 
    ON addresses.city_id = cities.id
    WHERE cities.name = 'Tula'""")

cur.execute(q)
data = cur.fetchall()
for item in data:
    print(' '.join(item))

Petr Petrov
Andrey Babkin
Masha Levitina


### 3. Число детей из Москвы
- JOIN
- Агрегация

In [23]:
q = ("""
    SELECT
    COUNT(children.id)
    FROM children
    JOIN addresses
    ON children.address_id = addresses.id
    JOIN cities 
    ON addresses.city_id = cities.id
    WHERE cities.name = 'Moscow'
    """)

cur.execute(q)
data = cur.fetchall()
print(data[0][0])

1


### 4. Вывести число заказов по каждому бренду 
- Where с объединением трёх таблиц в where

In [28]:
q = ("""
    SELECT brands.name
    , COUNT(*) AS `num`
    FROM children, gifts, brands
    WHERE children.gift_id = gifts.id and brands.id = gifts.brand_id
    GROUP BY name
    """)

cur.execute(q)
data = cur.fetchall()
for item, freq in data:
    print(item, freq, sep=': ')

Apple: 2
Hamleys: 1
Lego: 2


### 5. Все города, в которых живут дети 
- вложенный SELECT

В таблице cities указано больше городов, чем использовано в addresses.

In [64]:
q = ("""SELECT name 
        FROM cities 
        WHERE id IN 
            (SELECT city_id FROM children
            JOIN addresses
            ON children.address_id = addresses.id
            );""")
cur.execute(q)
data = cur.fetchall()
for item in data:
    print(item[0])

Moscow
Tula
Voronezh
